# **Notebook zur Evaluation - Trajektorien**
***

In [2250]:
import csv
import numpy as np
from scipy.spatial.transform import Rotation as R
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
import matplotlib.cm as cm
import plotly.graph_objects as go
import os
import re
import pandas as pd
from shapely.geometry import Polygon

***
### **Trajektorien CSVs laden**

In [2251]:
hook_num = 23
v = 10
hook_model = 'A'

frame = '_work'
# frame = ''

teil = 'Bauteil'

num_interp_points = 101
interp_step_size = 0.1

hole_width = 8
hole_height = 12

hook_width = 1.8
hook_height = 7.5

In [2252]:
model_a_widths = [1.6, 1.5, 1.7, 1.6, 1.9, 1.5, 1.7, 1.5, 1.4, 1.4, 1.5, 1.6, 1.6, 1.6, 1.6, 1.7, 1.7, 1.6, 1.7, 1.7, 1.7, 1.9, 1.9]
model_a_heights = [3.9, 4, 4.1, 4.2, 3.1, 3.6, 3.6, 4.1, 3.3, 3.5, 4.3, 4.2, 4.2, 4.4, 4.1, 4.8, 4.6, 4.9, 5.1, 5, 5, 5.2, 5.4]
model_b_widths = model_a_widths
model_b_heights = model_a_heights
model_c_widths = model_a_widths
model_c_heights = model_a_heights
model_d_widths = model_a_widths
model_d_heights = model_a_heights

In [2253]:
if hook_model == 'A':
    widths = model_a_widths
    heights = model_a_heights
if hook_model == 'B':
    widths = model_b_widths
    heights = model_b_heights
if hook_model == 'C':
    widths = model_c_widths
    heights = model_c_heights
if hook_model == 'D':
    widths = model_d_widths
    heights = model_d_heights

hook_width = widths[hook_num - 1]
hook_height = heights[hook_num - 1]

In [2254]:
def load_csv_to_trajectory(filepath):
    """Lädt die CSV und stellt Trajektorie-Liste wieder her"""
    trajectory = []
    with open(filepath, mode='r', newline='') as file:
        reader = csv.reader(file)
        next(reader)  # Überspringe Kopfzeile

        for row in reader:
            numbers = list(map(float, row))
            trans = numbers[:3]
            rot = numbers[3:]
            trajectory.append((trans, rot))
    return trajectory

In [2255]:
traj_optim_path = '/home/mo/Thesis/Evaluation/Trajektorientests/Modell' + hook_model + '/' + teil + '/v_' + str(v) + '/' + str(hook_num) + '/trajectory_0' + str(frame) + '.csv'
traj_1_path = '/home/mo/Thesis/Evaluation/Trajektorientests/Modell' + hook_model + '/' + teil + '/v_' + str(v) + '/' + str(hook_num) + '/trajectory_1' + str(frame) + '.csv'
traj_2_path = '/home/mo/Thesis/Evaluation/Trajektorientests/Modell' + hook_model + '/' + teil + '/v_' + str(v) + '/' + str(hook_num) + '/trajectory_2' + str(frame) + '.csv'
traj_3_path = '/home/mo/Thesis/Evaluation/Trajektorientests/Modell' + hook_model + '/' + teil + '/v_' + str(v) + '/' + str(hook_num) + '/trajectory_3' + str(frame) + '.csv'
traj_4_path = '/home/mo/Thesis/Evaluation/Trajektorientests/Modell' + hook_model + '/' + teil + '/v_' + str(v) + '/' + str(hook_num) + '/trajectory_4' + str(frame) + '.csv'

In [2256]:
trajectory_optim = load_csv_to_trajectory(traj_optim_path)
trajectory_1 = load_csv_to_trajectory(traj_1_path)
trajectory_2 = load_csv_to_trajectory(traj_2_path)
trajectory_3 = load_csv_to_trajectory(traj_3_path)
trajectory_4 = load_csv_to_trajectory(traj_4_path)

***
### **Interpolation der Trajektorie**

In [2257]:
def interpolate_trajectory(traj, num_points):
    """
    Interpoliert eine Trajektorie auf `num_points` gleichmäßig verteilte Punkte.
    Nur die translatorischen Komponenten werden verwendet (3D Punkte).
    """
    positions = np.array([p[0] for p in traj])  # Nur Positionen
    rotations = np.array([p[1] for p in traj])  # Nur Rotationen

    # Abstände zwischen aufeinanderfolgenden Punkten
    deltas_trans = np.linalg.norm(np.diff(positions, axis=0), axis=1)
    cumulative_dist_trans = np.concatenate([[0], np.cumsum(deltas_trans)])
    target_distances_trans = np.linspace(0, cumulative_dist_trans[-1], num_points)

    interp_positions = []
    for dist in target_distances_trans:
        idx = np.searchsorted(cumulative_dist_trans, dist) - 1
        idx = np.clip(idx, 0, len(positions) - 2)
        t = (dist - cumulative_dist_trans[idx]) / (cumulative_dist_trans[idx + 1] - cumulative_dist_trans[idx])
        point = (1 - t) * positions[idx] + t * positions[idx + 1]
        interp_positions.append(point)

    # Rotationsteil
    if np.allclose(rotations, rotations[0]):  # wenn alle Rotationswerte gleich sind (Ansatz 2-4)
        interp_rotations = [rotations[0]] * num_points
    else:
        deltas_rot = np.linalg.norm(np.diff(rotations, axis=0), axis=1)
        cumulative_dist_rot = np.concatenate([[0], np.cumsum(deltas_rot)])
        target_distances_rot = np.linspace(0, cumulative_dist_rot[-1], num_points)

        interp_rotations = []
        for rot in target_distances_rot:
            idx = np.searchsorted(cumulative_dist_rot, rot) - 1
            idx = np.clip(idx, 0, len(rotations) - 2)
            denom = cumulative_dist_rot[idx + 1] - cumulative_dist_rot[idx]
            t = 0 if denom == 0 else (rot - cumulative_dist_rot[idx]) / denom
            point = (1 - t) * rotations[idx] + t * rotations[idx + 1]
            interp_rotations.append(point)
    return np.array(interp_positions), np.array(interp_rotations)


def interpolate_trajectory_by_distance(traj, step_size):
    """
    Interpoliert eine Trajektorie ab Punkt 1 mit gleichmäßigem Abstand (step_size), 
    wobei Punkt 0 unverändert erhalten bleibt.
    """
    positions = np.array([p[0] for p in traj])
    rotations = np.array([p[1] for p in traj])

    # Segment ab Punkt 1
    positions_tail = positions[1:]
    rotations_tail = rotations[1:]

    deltas_trans = np.linalg.norm(np.diff(positions_tail, axis=0), axis=1)
    cumulative_dist_trans = np.concatenate([[0], np.cumsum(deltas_trans)])
    total_length = cumulative_dist_trans[-1]

    # Punkte ab Punkt 1 in step_size-Abständen
    target_distances_trans = np.arange(0, total_length + step_size, step_size)
    if target_distances_trans[-1] > total_length:
        target_distances_trans[-1] = total_length

    interp_positions = [positions[0]]  # Originalpunkt 0
    for dist in target_distances_trans:
        idx = np.searchsorted(cumulative_dist_trans, dist) - 1
        idx = np.clip(idx, 0, len(positions_tail) - 2)
        t = (dist - cumulative_dist_trans[idx]) / (cumulative_dist_trans[idx + 1] - cumulative_dist_trans[idx])
        point = (1 - t) * positions_tail[idx] + t * positions_tail[idx + 1]
        interp_positions.append(point)

    # Interpolation der Rotationen ab Punkt 1
    interp_rotations = [rotations[0]]  # Rotation für Punkt 0
    if np.allclose(rotations_tail, rotations_tail[0]):
        interp_rotations += [rotations_tail[0]] * (len(interp_positions) - 1)
    else:
        deltas_rot = np.linalg.norm(np.diff(rotations_tail, axis=0), axis=1)
        cumulative_dist_rot = np.concatenate([[0], np.cumsum(deltas_rot)])
        target_distances_rot = np.linspace(0, cumulative_dist_rot[-1], len(interp_positions) - 1)

        for rot in target_distances_rot:
            idx = np.searchsorted(cumulative_dist_rot, rot) - 1
            idx = np.clip(idx, 0, len(rotations_tail) - 2)
            denom = cumulative_dist_rot[idx + 1] - cumulative_dist_rot[idx]
            t = 0 if denom == 0 else (rot - cumulative_dist_rot[idx]) / denom
            point = (1 - t) * rotations_tail[idx] + t * rotations_tail[idx + 1]
            interp_rotations.append(point)
    return np.array(interp_positions), np.array(interp_rotations)


def match_trajectory_length(reference_traj, target_traj):
    """
    Passt die Länge der target_traj an die Länge der reference_traj an:
    - Wenn target_traj länger ist -> abschneiden.
    - Wenn target_traj kürzer ist -> letzten Punkt anhängen, bis gleich lang.
    """
    ref_len = len(reference_traj)
    tgt_len = len(target_traj)

    if tgt_len > ref_len:
        return target_traj[:ref_len]
    elif tgt_len < ref_len:
        last_pose = target_traj[-1]
        extra = [last_pose] * (ref_len - tgt_len)
        return target_traj + extra
    else:
        return target_traj


def compute_rectangle_corners(trajectory, width=8, height=12):
    rectangles = []

    half_w = width / 2
    half_h = height / 2

    # Rechteck immer in der XZ-Ebene
    local_corners = np.array([
        [-half_w, 0, -half_h],      # links unten
        [ half_w, 0, -half_h],      # rechts unten
        [ half_w, 0,  half_h],      # rechts oben
        [-half_w, 0,  half_h]       # links oben
    ])

    for pos, rpy in trajectory:
        rot = R.from_euler('xyz', rpy, degrees=True)
        rotated_corners = rot.apply(local_corners)
        translated_corners = rotated_corners + np.array(pos)
        rectangles.append(translated_corners.tolist())
    return rectangles

In [2258]:
# Interpolation für alle Trajektorien
def interpolate_and_package(traj):
    interp_pos, interp_rot = interpolate_trajectory_by_distance(traj, interp_step_size)
    return list(zip(interp_pos, interp_rot))

trajectory_optim = interpolate_and_package(trajectory_optim)
trajectory_1 = interpolate_and_package(trajectory_1)
trajectory_2 = interpolate_and_package(trajectory_2)
trajectory_3 = interpolate_and_package(trajectory_3)
trajectory_4 = interpolate_and_package(trajectory_4)

# Optimale Trajektorie an jede berechnete Trajektorie anpassen
trajectory_optim_1 = match_trajectory_length(trajectory_1, trajectory_optim)
trajectory_optim_2 = match_trajectory_length(trajectory_2, trajectory_optim)
trajectory_optim_3 = match_trajectory_length(trajectory_3, trajectory_optim)
trajectory_optim_4 = match_trajectory_length(trajectory_4, trajectory_optim)

# Rechtecke berechnen
rectangles_optim_hooks_1 = compute_rectangle_corners(trajectory_optim_1, width=hook_width, height=hook_height)
rectangles_optim_hooks_2 = compute_rectangle_corners(trajectory_optim_2, width=hook_width, height=hook_height)
rectangles_optim_hooks_3 = compute_rectangle_corners(trajectory_optim_3, width=hook_width, height=hook_height)
rectangles_optim_hooks_4 = compute_rectangle_corners(trajectory_optim_4, width=hook_width, height=hook_height)

rectangles_1 = compute_rectangle_corners(trajectory_1, width=hole_width, height=hole_height)
rectangles_2 = compute_rectangle_corners(trajectory_2, width=hole_width, height=hole_height)
rectangles_3 = compute_rectangle_corners(trajectory_3, width=hole_width, height=hole_height)
rectangles_4 = compute_rectangle_corners(trajectory_4, width=hole_width, height=hole_height)

rectangles_optim_1 = compute_rectangle_corners(trajectory_optim_1, width=hole_width, height=hole_height)
rectangles_optim_2 = compute_rectangle_corners(trajectory_optim_2, width=hole_width, height=hole_height)
rectangles_optim_3 = compute_rectangle_corners(trajectory_optim_3, width=hole_width, height=hole_height)
rectangles_optim_4 = compute_rectangle_corners(trajectory_optim_4, width=hole_width, height=hole_height)

In [2259]:
print("Kontrolle - Datenlänge der Trajektorie, Optim-Trajektore, Rechtecke, Optim Rechtecke, Haken-Rechtecke\n")
print("Trajektorie 1:")
print(f"Passt - {len(trajectory_1)} Daten-Punkte\n") if(len(trajectory_1) == len(trajectory_optim_1) == len(rectangles_1) == len(rectangles_optim_1) == len(rectangles_optim_hooks_1)) else ("Passt nicht!\n")

print("Trajektorie 2:")
print(f"Passt - {len(trajectory_2)} Daten-Punkte\n") if(len(trajectory_2) == len(trajectory_optim_2) == len(rectangles_2) == len(rectangles_optim_2) == len(rectangles_optim_hooks_2)) else ("Passt nicht!\n")

print("Trajektorie 3:")
print(f"Passt - {len(trajectory_3)} Daten-Punkte\n") if(len(trajectory_3) == len(trajectory_optim_3) == len(rectangles_3) == len(rectangles_optim_3) == len(rectangles_optim_hooks_3)) else ("Passt nicht!\n")

print("Trajektorie 4:")
print(f"Passt - {len(trajectory_4)} Daten-Punkte\n") if(len(trajectory_4) == len(trajectory_optim_4) == len(rectangles_4) == len(rectangles_optim_4) == len(rectangles_optim_hooks_4)) else ("Passt nicht!\n")

Kontrolle - Datenlänge der Trajektorie, Optim-Trajektore, Rechtecke, Optim Rechtecke, Haken-Rechtecke

Trajektorie 1:
Passt - 137 Daten-Punkte

Trajektorie 2:
Passt - 137 Daten-Punkte

Trajektorie 3:
Passt - 137 Daten-Punkte

Trajektorie 4:
Passt - 52 Daten-Punkte



***
### **Berechnung der Abweichungen zwischen Trajektorie und Optimalwerten**

In [2260]:
def absolute_traj_differences(traj_optim, traj):
    measurements = {}
    
    # Globale Diffs
    x_diffs, y_diffs, z_diffs = [], [], []
    roll_diffs, pitch_diffs, yaw_diffs = [], [], []
    euclidean_diffs = []

    # Lokale Diffs
    x_diffs_local, y_diffs_local, z_diffs_local = [], [], []

    if len(traj_optim) != len(traj):
        print("ERROR - Length of trajectories unequal!")
    else:
        for idx in range(len(traj_optim)):
            pos1, rpy1 = traj_optim[idx]
            pos2, rpy2 = traj[idx]

            pos1 = np.array(pos1)
            pos2 = np.array(pos2)
            rot1 = np.array(rpy1)
            rot2 = np.array(rpy2)

            pos_diff = pos2 - pos1
            rot_diff = rot2 - rot1

            # globale Differenzen
            euclidean_diffs.append(np.linalg.norm(pos_diff))
            x_diffs.append(pos_diff[0])
            y_diffs.append(pos_diff[1])
            z_diffs.append(pos_diff[2])
            roll_diffs.append(rot_diff[0])
            pitch_diffs.append(rot_diff[1])
            yaw_diffs.append(rot_diff[2])

            # lokale Transformierung
            r = R.from_euler('xyz', rpy1, degrees=True)

            local_diff = r.inv().apply(pos_diff)
            x_diffs_local.append(local_diff[0])
            y_diffs_local.append(local_diff[1])
            z_diffs_local.append(local_diff[2])

    # globale Einträge
    measurements['x_diffs'] = x_diffs
    measurements['y_diffs'] = y_diffs
    measurements['z_diffs'] = z_diffs
    measurements['roll_diffs'] = roll_diffs
    measurements['pitch_diffs'] = pitch_diffs
    measurements['yaw_diffs'] = yaw_diffs
    measurements['euclidean_diffs'] = euclidean_diffs

    measurements['x_min'] = np.min(x_diffs)
    measurements['x_max'] = np.max(x_diffs)
    measurements['y_min'] = np.min(y_diffs)
    measurements['y_max'] = np.max(y_diffs)
    measurements['z_min'] = np.min(z_diffs)
    measurements['z_max'] = np.max(z_diffs)

    measurements['x_diffs_mean'] = np.mean(x_diffs)
    measurements['y_diffs_mean'] = np.mean(y_diffs)
    measurements['z_diffs_mean'] = np.mean(z_diffs)

    # lokale Einträge
    measurements['x_diffs_local'] = x_diffs_local
    measurements['y_diffs_local'] = y_diffs_local
    measurements['z_diffs_local'] = z_diffs_local

    measurements['x_min_local'] = np.min(x_diffs_local)
    measurements['x_max_local'] = np.max(x_diffs_local)
    measurements['y_min_local'] = np.min(y_diffs_local)
    measurements['y_max_local'] = np.max(y_diffs_local)
    measurements['z_min_local'] = np.min(z_diffs_local)
    measurements['z_max_local'] = np.max(z_diffs_local)

    measurements['x_diffs_mean_local'] = np.mean(x_diffs_local)
    measurements['y_diffs_mean_local'] = np.mean(y_diffs_local)
    measurements['z_diffs_mean_local'] = np.mean(z_diffs_local)
    return measurements

In [2261]:
traj1_measurements = absolute_traj_differences(traj_optim = trajectory_optim_1, traj = trajectory_1)
traj2_measurements = absolute_traj_differences(traj_optim = trajectory_optim_2, traj = trajectory_2)
traj3_measurements = absolute_traj_differences(traj_optim = trajectory_optim_3, traj = trajectory_3)
traj4_measurements = absolute_traj_differences(traj_optim = trajectory_optim_4, traj = trajectory_4)

In [2262]:
def plot_position_differences(measurements, hook_num, trajectory_process=0):
    # Globale Werte
    x = measurements['x_diffs']
    y = measurements['y_diffs']
    z = measurements['z_diffs']
    x_diffs_mean = measurements['x_diffs_mean']
    y_diffs_mean = measurements['y_diffs_mean']
    z_diffs_mean = measurements['z_diffs_mean']

    # Lokale Werte
    x_loc = measurements['x_diffs_local']
    y_loc = measurements['y_diffs_local']
    z_loc = measurements['z_diffs_local']
    x_loc_mean = measurements['x_diffs_mean_local']
    y_loc_mean = measurements['y_diffs_mean_local']
    z_loc_mean = measurements['z_diffs_mean_local']

    n = len(x)
    t = list(range(n))

    fig, axs = plt.subplots(2, 1, figsize=(14, 10))
    fig.suptitle(f"Positionsdifferenzen für Haken {hook_num} - Trajektorien-Ansatz {trajectory_process}", fontsize=16)

    ##### Subplot 1: Globale Differenzen
    axs[0].plot(t, x, label='x-Differenz', color='red')
    axs[0].axhline(x_diffs_mean, linestyle='--', color='red', label=rf"$\bar{{x}}$ = {x_diffs_mean:.3f}")
    axs[0].text(n * 0.2, x_diffs_mean + 0.05, rf"$\bar{{x}}$ = {x_diffs_mean:.3f}", color='red',
                fontsize=10, bbox=dict(facecolor='white', edgecolor='red', alpha=0.9))

    axs[0].plot(t, y, label='y-Differenz', color='green')
    axs[0].axhline(y_diffs_mean, linestyle='--', color='green', label=rf"$\bar{{y}}$ = {y_diffs_mean:.3f}")
    axs[0].text(n * 0.3, y_diffs_mean + 0.05, rf"$\bar{{y}}$ = {y_diffs_mean:.3f}", color='green',
                fontsize=10, bbox=dict(facecolor='white', edgecolor='green', alpha=0.9))

    axs[0].plot(t, z, label='z-Differenz', color='blue')
    axs[0].axhline(z_diffs_mean, linestyle='--', color='blue', label=rf"$\bar{{z}}$ = {z_diffs_mean:.3f}")
    axs[0].text(n * 0.4, z_diffs_mean + 0.05, rf"$\bar{{z}}$ = {z_diffs_mean:.3f}", color='blue',
                fontsize=10, bbox=dict(facecolor='white', edgecolor='blue', alpha=0.9))

    axs[0].set_title("Globale Positionsabweichung")
    axs[0].set_xlabel('Trajektorienpunkt')
    axs[0].set_ylabel('Differenz [mm]')
    axs[0].set_xlim(0, len(x))
    axs[0].legend()
    axs[0].grid(True)

    ##### Subplot 2: Lokale Differenzen
    axs[1].plot(t, x_loc, label='x-Differenz (lokal)', color='red')
    axs[1].axhline(x_loc_mean, linestyle=':', color='red', label=rf"$\bar{{x}}$ = {x_loc_mean:.3f}")
    axs[1].text(n * 0.2, x_loc_mean + 0.05, rf"$\bar{{x}}$ = {x_loc_mean:.3f}", color='red',
                fontsize=10, bbox=dict(facecolor='white', edgecolor='red', alpha=0.9))

    axs[1].plot(t, y_loc, label='y-Differenz (lokal)', color='green')
    axs[1].axhline(y_loc_mean, linestyle=':', color='green', label=rf"$\bar{{y}}$ = {y_loc_mean:.3f}")
    axs[1].text(n * 0.3, y_loc_mean + 0.05, rf"$\bar{{y}}$ = {y_loc_mean:.3f}", color='green',
                fontsize=10, bbox=dict(facecolor='white', edgecolor='green', alpha=0.9))

    axs[1].plot(t, z_loc, label='z-Differenz (lokal)', color='blue')
    axs[1].axhline(z_loc_mean, linestyle=':', color='blue', label=rf"$\bar{{z}}$ = {z_loc_mean:.3f}")
    axs[1].text(n * 0.4, z_loc_mean + 0.05, rf"$\bar{{z}}$ = {z_loc_mean:.3f}", color='blue',
                fontsize=10, bbox=dict(facecolor='white', edgecolor='blue', alpha=0.9))
    
    delta_width = (hole_width - hook_width) / 2
    delta_height = (hole_height - hole_width) / 2

    axs[1].plot(t, [delta_width] * len(x), linestyle='--', color='red', label='Kollisionsgrenze entlang X')
    axs[1].plot(t, [-delta_width] * len(x), linestyle='--', color='red')
    axs[1].plot(t, [delta_height] * len(x), linestyle='--', color='blue', label='Kollisionsgrenze entlang Z')
    axs[1].plot(t, [-delta_height] * len(x), linestyle='--', color='blue')

    axs[1].set_title("Lokale Positionsabweichung")
    axs[1].set_xlabel('Trajektorienpunkt')
    axs[1].set_ylabel('Differenz [mm]')
    axs[1].set_xlim(0, len(x))
    axs[1].legend()
    axs[1].grid(True)

    plt.tight_layout(rect=[0, 0.03, 1, 0.95])
    plt.show()

In [2263]:
def show_and_save_measurements(dict, hook_num, trajectory_process=0, savepath='/home/mo/Thesis/Evaluation/Trajektorientests/CSV-Exporte/Frame_' + str(frame), output=False):
    if output:
        print("-------------------------------------------------------------")
        print(f"Messwerte für Trajektorie - Haken {hook_num} - Trajektorien-Ansatz {trajectory_process}\n")
        print("Absolute Distanzen\n")
        print(f"x max [WORK]: {dict['x_max']:.4f}mm - x max [LOCAL]: {dict['x_max_local']:.4f}mm")
        print(f"x min [WORK]: {dict['x_min']:.4f}mm - x min [LOCAL]: {dict['x_min_local']:.4f}mm")
        print(f"y max [WORK]: {dict['y_max']:.4f}mm - y max [LOCAL]: {dict['y_max_local']:.4f}mm")
        print(f"y min [WORK]: {dict['y_min']:.4f}mm - y min [LOCAL]: {dict['y_min_local']:.4f}mm")
        print(f"z max [WORK]: {dict['z_max']:.4f}mm - z max [LOCAL]: {dict['z_max_local']:.4f}mm")
        print(f"z min [WORK]: {dict['z_min']:.4f}mm - z min [LOCAL]: {dict['z_min_local']:.4f}mm\n")
    
        print(f"x mean [WORK]: {dict['x_diffs_mean']:.4f}mm - x mean [LOCAL]: {dict['x_diffs_mean_local']:.4f}mm")
        print(f"y mean [WORK]: {dict['y_diffs_mean']:.4f}mm - y mean [LOCAL]: {dict['y_diffs_mean_local']:.4f}mm")
        print(f"z mean [WORK]: {dict['z_diffs_mean']:.4f}mm - z mean [LOCAL]: {dict['z_diffs_mean_local']:.4f}mm\n")
    
        print()
        print(f"Maximale Euklidsche Distanz [WORK]: {np.max(dict['euclidean_diffs']):.4f}\n")
        
        print()
        print("Roll-Pitch-Yaw Abweichungen\n")
        print(f"Maximale Abweichung Roll (x) [WORK]: {np.max(dict['roll_diffs']):.4f}")
        print(f"Maximale Abweichung Pitch (y) [WORK]: {np.max(dict['pitch_diffs']):.4f}°")
        print(f"Maximale Abweichung Yaw (z) [WORK]: {np.max(dict['yaw_diffs']):.4f}°")

    # Daten in CSV speichern
    data = {
        "hook_num": hook_num,
        "trajectory_process": trajectory_process,
        "x_max_work": dict['x_max'],
        "x_max_local": dict['x_max_local'],
        "x_min_work": dict['x_min'],
        "x_min_local": dict['x_min_local'],
        "y_max_work": dict['y_max'],
        "y_max_local": dict['y_max_local'],
        "y_min_work": dict['y_min'],
        "y_min_local": dict['y_min_local'],
        "z_max_work": dict['z_max'],
        "z_max_local": dict['z_max_local'],
        "z_min_work": dict['z_min'],
        "z_min_local": dict['z_min_local'],
        "x_mean_work": dict['x_diffs_mean'],
        "x_mean_local": dict['x_diffs_mean_local'],
        "y_mean_work": dict['y_diffs_mean'],
        "y_mean_local": dict['y_diffs_mean_local'],
        "z_mean_work": dict['z_diffs_mean'],
        "z_mean_local": dict['z_diffs_mean_local'],
        "euclidean_max_work": np.max(dict['euclidean_diffs']),
        "roll_max": np.max(dict['roll_diffs']),
        "pitch_max": np.max(dict['pitch_diffs']),
        "yaw_max": np.max(dict['yaw_diffs']),
    }

    df = pd.DataFrame([data])
    os.makedirs(savepath, exist_ok=True)
    filename = f"measurements_hook{hook_num}_traj{trajectory_process}.csv"
    full_path = os.path.join(savepath, filename)
    df.to_csv(full_path, index=False, float_format="%.15g")
    print(f"\nWerte als CSV gespeichert unter: {full_path}")

In [2264]:
# plot_position_differences(traj1_measurements, hook_num, trajectory_process = 1)
show_and_save_measurements(traj1_measurements, hook_num, trajectory_process = 1)

# plot_position_differences(traj2_measurements, hook_num, trajectory_process = 2)
show_and_save_measurements(traj2_measurements, hook_num, trajectory_process = 2)

# plot_position_differences(traj3_measurements, hook_num, trajectory_process = 3)
show_and_save_measurements(traj3_measurements, hook_num, trajectory_process = 3)

# plot_position_differences(traj4_measurements, hook_num, trajectory_process = 4)
show_and_save_measurements(traj4_measurements, hook_num, trajectory_process = 4)


Werte als CSV gespeichert unter: /home/mo/Thesis/Evaluation/Trajektorientests/CSV-Exporte/Frame__work/measurements_hook23_traj1.csv

Werte als CSV gespeichert unter: /home/mo/Thesis/Evaluation/Trajektorientests/CSV-Exporte/Frame__work/measurements_hook23_traj2.csv

Werte als CSV gespeichert unter: /home/mo/Thesis/Evaluation/Trajektorientests/CSV-Exporte/Frame__work/measurements_hook23_traj3.csv

Werte als CSV gespeichert unter: /home/mo/Thesis/Evaluation/Trajektorientests/CSV-Exporte/Frame__work/measurements_hook23_traj4.csv


***
### **Kollisionsberechnung**

In [2265]:
def get_plane_from_rectangle(rect):
    """
    Erzeugt eine Ebene aus einem Rechteck: Rückgabe ist (Punkt, Normalenvektor)
    """
    p0 = np.array(rect[0])
    p1 = np.array(rect[1])
    p3 = np.array(rect[3])
    normal = np.cross(p1 - p0, p3 - p0)
    normal /= np.linalg.norm(normal)
    return p0, normal


def line_plane_intersection(p0, p1, plane_point, plane_normal):
    """
    Schnitt einer Linie (p0 → p1) mit einer Ebene (plane_point, plane_normal).
    Gibt (t, Punkt) zurück, wenn Schnitt im Segment liegt (0 ≤ t ≤ 1), sonst None.
    """
    u = p1 - p0
    w = p0 - plane_point
    denom = np.dot(plane_normal, u)
    
    if abs(denom) < 1e-8:
        return None  # Parallel

    t = -np.dot(plane_normal, w) / denom
    if 0 <= t <= 1:
        return t, p0 + t * u
    return None


def find_intersection_with_plane_on_trajectory(rectangles, plane_point, plane_normal):
    """
    Geht Rechtecke durch und sucht den Schnittpunkt der Mittellinie mit der Ebene.
    Gibt (Index, t, Punkt) zurück.
    """
    centers = [np.mean(rect, axis=0) for rect in rectangles]
    for i in range(len(centers) - 1):
        result = line_plane_intersection(centers[i], centers[i+1], plane_point, plane_normal)
        if result:
            t, intersection = result
            return i, t, intersection
    return None


def interpolate_rectangle(rect1, rect2, t):
    """
    Interpoliert zwei Rechtecke (je 4 Punkte) linear mit Faktor t ∈ [0, 1]
    """
    rect_interp = []
    for p1, p2 in zip(rect1, rect2):
        p_interp = (1 - t) * np.array(p1) + t * np.array(p2)
        rect_interp.append(p_interp)
    return rect_interp


def transform_hook_to_hole_frame(hook_rect, hole_rect):
    """
    Transformiert ein Hakenrechteck in das lokale Koordinatensystem eines Lochrechtecks.
    Rückgabe: Punkte des Hakenrechtecks in der XZ-Ebene des Loch-KS.
    """
    # Punkte des Lochrechtecks
    p0, p1, p2, p3 = [np.array(p) for p in hole_rect]

    # X-Achse: kurze Seite → von p0 nach p1
    x_axis = p1 - p0
    x_axis /= np.linalg.norm(x_axis)

    # Z-Achse: lange Seite → von p0 nach p3
    z_axis = p3 - p0
    z_axis /= np.linalg.norm(z_axis)

    # Y-Achse: rechtshändig
    y_axis = np.cross(z_axis, x_axis)
    y_axis /= np.linalg.norm(y_axis)

    # Lokales KS (R: Spalten = Achsen, T: Ursprung im Mittelpunkt)
    R = np.stack([x_axis, y_axis, z_axis], axis=1)

    # Ursprung im Mittelpunkt des Lochrechtecks (optional: p0 + diagonale/2)
    origin = np.mean([p0, p1, p2, p3], axis=0)

    # Transformation der Hakenpunkte
    hook_rect_local = []
    for point in hook_rect:
        local = R.T @ (np.array(point) - origin)  # Rotation + Verschiebung
        hook_rect_local.append((local[0], local[2]))  # Nur XZ-Koordinaten
    return hook_rect_local


def check_hook_collides_in_hole(hook_rect, loch_rect, epsilon=1e-6):
    """
    Prüft, ob das Hakenrechteck das Langloch verlässt oder berührt, im lokalen KS des Lochs.
    """
    hook_local = transform_hook_to_hole_frame(hook_rect, loch_rect)
    
    # Loch als Rechteck in XZ (±W/2 × ±H/2)
    loch_width = np.linalg.norm(np.array(loch_rect[1]) - np.array(loch_rect[0]))
    loch_height = np.linalg.norm(np.array(loch_rect[3]) - np.array(loch_rect[0]))
    hw, hh = loch_width / 2, loch_height / 2

    loch_poly = Polygon([[-hw, -hh], [hw, -hh], [hw, hh], [-hw, hh]])
    hook_poly = Polygon(hook_local)
    return not loch_poly.buffer(-epsilon).contains(hook_poly)


def evaluate_collisions_via_plane_intersections(hook_rects_opt, loch_rects_calc):
    collisions = []
    rectangles_collisions = []

    for i, hook_rect in enumerate(hook_rects_opt):
        plane_point, plane_normal = get_plane_from_rectangle(hook_rect)
        result = find_intersection_with_plane_on_trajectory(loch_rects_calc, plane_point, plane_normal)
        if result:
            idx, t, _ = result
            loch_rect_interp = interpolate_rectangle(loch_rects_calc[idx], loch_rects_calc[idx + 1], t)
            if check_hook_collides_in_hole(hook_rect, loch_rect_interp):
                collisions.append(i)
                rectangles_collisions.append(loch_rect_interp)
    return collisions, rectangles_collisions

In [2266]:
collisions_1, rectangles_collisions_1 = evaluate_collisions_via_plane_intersections(rectangles_optim_hooks_1, rectangles_1)
print(f"Trajektorie 1: {len(collisions_1)} Kollisionen gefunden.")

collisions_2, rectangles_collisions_2 = evaluate_collisions_via_plane_intersections(rectangles_optim_hooks_2, rectangles_2)
print(f"Trajektorie 1: {len(collisions_2)} Kollisionen gefunden.")

collisions_3, rectangles_collisions_3 = evaluate_collisions_via_plane_intersections(rectangles_optim_hooks_3, rectangles_3)
print(f"Trajektorie 1: {len(collisions_3)} Kollisionen gefunden.")

collisions_4, rectangles_collisions_4 = evaluate_collisions_via_plane_intersections(rectangles_optim_hooks_4, rectangles_4)
print(f"Trajektorie 1: {len(collisions_4)} Kollisionen gefunden.")

Trajektorie 1: 136 Kollisionen gefunden.
Trajektorie 1: 136 Kollisionen gefunden.
Trajektorie 1: 136 Kollisionen gefunden.
Trajektorie 1: 51 Kollisionen gefunden.


In [2267]:
'''
idx = 100
traj = rectangles_2
hook_local = transform_hook_to_hole_frame(rectangles_optim_hooks_1[idx], traj[idx])

loch_width = np.linalg.norm(np.array(traj[idx][1]) - np.array(traj[idx][0]))
loch_height = np.linalg.norm(np.array(traj[idx][3]) - np.array(traj[idx][0]))
hw, hh = loch_width / 2, loch_height / 2

fig, ax = plt.subplots()
ax.add_patch(plt.Rectangle((-hw, -hh), 2*hw, 2*hh,
                           edgecolor='r', facecolor='none', label='Loch', linewidth=2))

hook = np.array(hook_local + [hook_local[0]])  # schließen
ax.plot(hook[:, 0], hook[:, 1], 'b-o', label='Haken')

ax.set_aspect('equal')
ax.grid()
ax.legend()
ax.set_title(f"Kollisionscheck Step {idx}")
plt.show()
'''

'\nidx = 100\ntraj = rectangles_2\nhook_local = transform_hook_to_hole_frame(rectangles_optim_hooks_1[idx], traj[idx])\n\nloch_width = np.linalg.norm(np.array(traj[idx][1]) - np.array(traj[idx][0]))\nloch_height = np.linalg.norm(np.array(traj[idx][3]) - np.array(traj[idx][0]))\nhw, hh = loch_width / 2, loch_height / 2\n\nfig, ax = plt.subplots()\nax.add_patch(plt.Rectangle((-hw, -hh), 2*hw, 2*hh,\n                           edgecolor=\'r\', facecolor=\'none\', label=\'Loch\', linewidth=2))\n\nhook = np.array(hook_local + [hook_local[0]])  # schließen\nax.plot(hook[:, 0], hook[:, 1], \'b-o\', label=\'Haken\')\n\nax.set_aspect(\'equal\')\nax.grid()\nax.legend()\nax.set_title(f"Kollisionscheck Step {idx}")\nplt.show()\n'

***
### **3D-Plot der Trajektorie und Export als HTML**

In [2268]:
def visualize_rectangles_html(rectangles, connect_points=None, 
                              secondary_rectangles=None, secondary_legend_title=None,
                              hook_rectangles = None,
                              hook_num=0, hook_model = 'A', trajectory_process=1, html_filename="/home/mo/Thesis/Evaluation/Trajektorientests/CSV/plot_interaktiv.html",
                              collision_list = None, rectangles_collisions = None, plot_collisions = False):
    """
    Visualisiert Rechtecke und Trajektorien im 3D-Raum als interaktive HTML-Datei.
        -> rectangles: Liste von Rechtecken (jeweils 4 Punkte [x, y, z])
        -> figsize: Größe der Plotfläche (Breite, Höhe)
        -> connect_points: Optionales Tupel mit zwei Punkten [x, y, z], die verbunden werden sollen
        -> secondary_rectangles: Optionaler zweiter Satz Rechtecke (werden grau dargestellt)
        -> secondary_legend_title: Optionaler Legenden-Eintrag für die grauen Rechtecke
        -> html_filename: Der Name der HTML-Datei, die erstellt wird
    """
    fig = go.Figure()

    ##### Berechnete Trajektorie (farbig)
    cmap = plt.cm.get_cmap('tab20')
    centers = []
    if rectangles:
        for i, rect in enumerate(rectangles):
            # Verwende rgba-String für die Farbe (anstatt des RGBA-Tupels)
            '''
            color = f"rgba({int(cmap(i / len(rectangles))[0] * 255)}, " \
                    f"{int(cmap(i / len(rectangles))[1] * 255)}, " \
                    f"{int(cmap(i / len(rectangles))[2] * 255)}, 1.0)"
            '''
            color = 'blue'
            x, y, z = zip(*rect)

            fig.add_trace(go.Mesh3d(x=x, y=y, z=z, color=color, opacity=0.05, 
                                   name=f'P{i+1}'))
        
            center = np.mean(np.array(rect), axis=0)
            centers.append(center)

            text_label = ''
            # Füge den Text an den Punkten hinzu (mit entsprechender Position)
            fig.add_trace(go.Scatter3d(x=[center[0]], y=[center[1]], z=[center[2]], 
                                       mode='markers+text', marker=dict(size=3, color='blue'),
                                       text=text_label, textposition="top center", textfont=dict(size=12), showlegend=False))
            
            # Hinzufügen der Umrandung der Rechtecke mit einer Linie
            if i == 0 or i== 1 or i == (len(rectangles) - 1 ):
                for j in range(4):
                    x0, y0, z0 = rect[j]
                    x1, y1, z1 = rect[(j + 1) % 4]  # Nächster Punkt im Rechteck
                    fig.add_trace(go.Scatter3d(x=[x0, x1], y=[y0, y1], z=[z0, z1], 
                                               mode='lines', 
                                               line=dict(color=color, width=4),  # Keine opacity hier
                                               showlegend=False))
        
        # Linie durch die berechneten Mittelpunkte
        centers = np.array(centers)
        fig.add_trace(go.Scatter3d(x=centers[:, 0], y=centers[:, 1], z=centers[:, 2],
                                   mode='lines', line=dict(color='black', width=2), name='Berechnete Hakenlinie'))

        # Beschriftung der ersten und letzten Punkte
        if len(centers) > 0:
            fig.add_trace(go.Scatter3d(x=[centers[0, 0]], y=[centers[0, 1]], z=[centers[0, 2]],
                                       mode='text', text=['Pre-Position'], textposition="top center", textfont=dict(size=10), showlegend=False))
        if len(centers) > 2:
            fig.add_trace(go.Scatter3d(x=[centers[-1, 0]+1], y=[centers[-1, 1]+1], z=[centers[-1, 2]+1],
                                       mode='text', text=['Loslassen'], textposition="top center", textfont=dict(size=10), showlegend=False))
        
        # Tunnelkanten: Ecken durchziehen mit denselben Farben wie bei den Rechtecken
        num_corners = 4  # Immer 4 Ecken pro Rechteck
        for corner_idx in range(num_corners):
            x_line, y_line, z_line = [], [], []
            colors = []
            for i, rect in enumerate(rectangles):
                # Farbcode für das Rechteck
                colors.append(f"rgba({int(cmap(i / len(rectangles))[0] * 255)}, " \
                        f"{int(cmap(i / len(rectangles))[1] * 255)}, " \
                        f"{int(cmap(i / len(rectangles))[2] * 255)}, 1.0)")
                            
                x, y, z = rect[corner_idx]
                x_line.append(x)
                y_line.append(y)
                z_line.append(z)
            
            # Die Tunnelkanten mit dem Farbcode der Rechtecke zeichnen
            fig.add_trace(go.Scatter3d(x=x_line, y=y_line, z=z_line,
                                       mode='lines',
                                       line=dict(color='blue', width=3),
                                       showlegend=False))


    ##### Optimal-Trajektorie (grün) – Zweite Rechtecke
    if secondary_rectangles:
        for i,rect in enumerate(secondary_rectangles):
            x, y, z = zip(*rect)
            fig.add_trace(go.Mesh3d(x=x, y=y, z=z, color='rgba(0,255,0,1.0)', opacity=0.05, 
                                   name=secondary_legend_title or 'Optimale Trajektorie'))
            
            # Hinzufügen der Umrandung der Rechtecke mit einer Linie
            if i == 0 or i == 1 or i == (len(secondary_rectangles) - 1):
                for j in range(4):
                    x0, y0, z0 = rect[j]
                    x1, y1, z1 = rect[(j + 1) % 4]  # Nächster Punkt im Rechteck
                    fig.add_trace(go.Scatter3d(x=[x0, x1], y=[y0, y1], z=[z0, z1], 
                                               mode='lines', 
                                               line=dict(color='rgba(0,255,0,1.0)', width=3),  # Keine opacity hier
                                               showlegend=False))
            
            # Hinzufügen eines roten Punktes in der Mitte des Rechtecks
            center = np.mean(np.array(rect), axis=0)
            fig.add_trace(go.Scatter3d(x=[center[0]], y=[center[1]], z=[center[2]], 
                                       mode='markers', marker=dict(size=2, color='green'), 
                                       showlegend=False))
        
        # Mittelpunkte der sekundären Rechtecke sammeln
        secondary_centers = [np.mean(np.array(rect), axis=0) for rect in secondary_rectangles]
        secondary_centers = np.array(secondary_centers)

        # Linie durch die Mittelpunkte zeichnen
        fig.add_trace(go.Scatter3d(
            x=secondary_centers[:, 0], y=secondary_centers[:, 1], z=secondary_centers[:, 2],
            mode='lines',
            line=dict(color='rgba(0,255,0,1.0)', width=2),
            name='Optimale Hakenlinie'
        ))

        # Tunnelkanten Optimal-Trajektorie
        num_corners = 4  # Immer 4 Ecken pro Rechteck
        for corner_idx in range(num_corners):
            x_line, y_line, z_line = [], [], []
            colors = []
            for i, rect in enumerate(secondary_rectangles):                       
                x, y, z = rect[corner_idx]
                x_line.append(x)
                y_line.append(y)
                z_line.append(z)

            # Die Tunnelkanten mit dem Farbcode der Rechtecke zeichnen
            fig.add_trace(go.Scatter3d(x=x_line, y=y_line, z=z_line,
                                       mode='lines',
                                       line=dict(color='rgba(0,255,0,1.0)', width=3),
                                       showlegend=False))


    ##### Modellierte Hakenlinie
    if hook_rectangles:
        for i, rect in enumerate(hook_rectangles):
            if i > 0:
                x, y, z = zip(*rect)
                show_legend = i == 1  # Nur beim ersten Eintrag die Legende zeigen
                fig.add_trace(go.Mesh3d(
                    x=x, y=y, z=z,
                    color='rgba(0,0,0,1.0)',
                    opacity=1.0,
                    name='Haken (modelliert)',
                    showlegend=show_legend
                ))
                
                # Umrandungslinien (optional showlegend=False hier)
                if i == 0 or i == (len(hook_rectangles) - 1):
                    for j in range(4):
                        x0, y0, z0 = rect[j]
                        x1, y1, z1 = rect[(j + 1) % 4]
                        fig.add_trace(go.Scatter3d(
                            x=[x0, x1], y=[y0, y1], z=[z0, z1],
                            mode='lines', line=dict(color='rgba(0,0,0,1.0)', width=3), showlegend=False
                        ))
                
                # Mittelpunkt der Senke (letztes Rechteck) und Spitze (erstes Rechteck) als Punkt markieren
                if i == len(hook_rectangles) - 1:
                    mx = sum(p[0] for p in rect) / 4
                    my = sum(p[1] for p in rect) / 4
                    mz = sum(p[2] for p in rect) / 4
        
                    fig.add_trace(go.Scatter3d(
                        x=[mx], y=[my], z=[mz],
                        mode='markers', marker=dict(size=8, color='cyan'), name='Senke (modelliert)'
                    ))
                    fig.add_trace(go.Scatter3d(
                        x=[mx], y=[my], z=[mz],
                        mode='text', text=['Senke (modelliert)'], textposition="top center", textfont=dict(size=10), showlegend=False)
                    )
                elif i == 1:
                    mx = sum(p[0] for p in rect) / 4
                    my = sum(p[1] for p in rect) / 4
                    mz = sum(p[2] for p in rect) / 4
        
                    fig.add_trace(go.Scatter3d(
                        x=[mx], y=[my], z=[mz],
                        mode='markers', marker=dict(size=8, color='cyan'), name='Spitze (modelliert)'
                    ))
                    fig.add_trace(go.Scatter3d(
                        x=[mx], y=[my], z=[mz],
                        mode='text', text=['Spitze (modelliert)'], textposition="top center", textfont=dict(size=10), showlegend=False)
                    )


    ##### Spitze und Senke (berechnet) in magenta inkl. graue Verbindungslinie
    if connect_points:
        p1, p2 = np.array(connect_points[0]), np.array(connect_points[1])
        fig.add_trace(go.Scatter3d(x=[p1[0], p2[0]], y=[p1[1], p2[1]], z=[p1[2], p2[2]], 
                                   mode='lines', line=dict(color='gray', dash='dash'), name='Direkte Gerade Spitze -> Senke'))
        fig.add_trace(go.Scatter3d(x=[p1[0]], y=[p1[1]], z=[p1[2]], mode='markers', 
                                   marker=dict(size=8, color='magenta'), name='Spitze (berechnet)'))
        fig.add_trace(go.Scatter3d(x=[p2[0]], y=[p2[1]], z=[p2[2]], mode='markers', 
                                   marker=dict(size=8, color='magenta'), name='Senke (berechnet)'))
        
        # Hinzufügen von Text "Spitze" und "Senke"
        fig.add_trace(go.Scatter3d(x=[p1[0]], y=[p1[1]], z=[p1[2]], mode='text', 
                                   text=['Spitze (berechnet)'], textposition="top center", textfont=dict(size=10), showlegend=False))
        fig.add_trace(go.Scatter3d(x=[p2[0]], y=[p2[1]], z=[p2[2]], mode='text', 
                                   text=['Senke (berechnet)'], textposition="top center", textfont=dict(size=10), showlegend=False))
        

    ##### Kollisionen in rot
    if plot_collisions == True and collision_list is not None:
        first_i_for_legend = None
        for i, rect in enumerate(hook_rectangles):      # wenn Index in den Kollisionen enthalten -> rot einzeichnen
            if i in collision_list:
                x, y, z = zip(*rect)
                if first_i_for_legend is None:
                    first_i_for_legend = True      # Legendeeintrag nur für ersten Index zeigen
                
                fig.add_trace(go.Mesh3d(
                    x=x, y=y, z=z,
                    color='red',
                    opacity=1.0,
                    name='Kollision',
                    showlegend=first_i_for_legend
                ))
                first_i_for_legend = False
                
                # Umrandungslinien (optional showlegend=False hier)
                if i == 0 or i == (len(hook_rectangles) - 1):
                    for j in range(4):
                        x0, y0, z0 = rect[j]
                        x1, y1, z1 = rect[(j + 1) % 4]
                        fig.add_trace(go.Scatter3d(
                            x=[x0, x1], y=[y0, y1], z=[z0, z1],
                            mode='lines',
                            line=dict(color='red', width=3),
                            showlegend=False  # keine doppelten Legenden für Linien
                        ))
        for i, rect in enumerate(rectangles_collisions):      # wenn Index in den Kollisionen enthalten -> rot einzeichnen
            x, y, z = zip(*rect)
            if first_i_for_legend is None:
                first_i_for_legend = True      # Legendeeintrag nur für ersten Index zeigen
            
            fig.add_trace(go.Mesh3d(
                x=x, y=y, z=z,
                color='red',
                opacity=1.0,
                name='Kollision',
                showlegend=first_i_for_legend
            ))
            first_i_for_legend = False
            
            # Umrandungslinien (optional showlegend=False hier)
            if i == 0 or i == (len(hook_rectangles) - 1):
                for j in range(4):
                    x0, y0, z0 = rect[j]
                    x1, y1, z1 = rect[(j + 1) % 4]
                    fig.add_trace(go.Scatter3d(
                        x=[x0, x1], y=[y0, y1], z=[z0, z1],
                        mode='lines',
                        line=dict(color='red', width=3),
                        showlegend=False  # keine doppelten Legenden für Linien
                    ))
        

    # Achsen und Layout
    fig.update_layout(
        scene=dict(
            xaxis_title='X',
            yaxis_title='Y',
            zaxis_title='Z',
            aspectmode='data',      # cube
            camera=dict(
                up=dict(x=0, y=-1., z=0),
                eye=dict(x=-0.3, y=-0.3, z=-2)
            )),
        title=f'Haken {hook_num} (Modell {hook_model}), Trajektorien-Ansatz {trajectory_process}',
        legend=dict(title="Legende"))
    

    # HTML exportieren
    fig.write_html(html_filename)
    print(f"HTML-Datei '{html_filename}' erfolgreich erstellt!")
    # fig.show()

In [2269]:
visualize_rectangles_html(
    rectangles = rectangles_1,
    secondary_rectangles = rectangles_optim_1,
    secondary_legend_title = 'optimale Trajektorie',
    hook_rectangles = rectangles_optim_hooks_1,
    connect_points = (trajectory_1[1][0], trajectory_1[-1][0]),
    hook_num = hook_num,
    hook_model = hook_model,
    trajectory_process = 1,
    html_filename = '/home/mo/Thesis/Evaluation/Trajektorientests/traj1_frame_' + str(frame) + '.html',
    collision_list = collisions_1, rectangles_collisions = rectangles_collisions_1, plot_collisions = True,
    )

visualize_rectangles_html(
    rectangles = rectangles_2,
    secondary_rectangles = rectangles_optim_2,
    secondary_legend_title = 'optimale Trajektorie',
    hook_rectangles = rectangles_optim_hooks_2,
    connect_points = (trajectory_2[1][0], trajectory_2[-1][0]),
    hook_num = hook_num,
    hook_model = hook_model,
    trajectory_process = 2,
    html_filename = '/home/mo/Thesis/Evaluation/Trajektorientests/traj2_frame_' + str(frame) + '.html',
    collision_list = collisions_2, rectangles_collisions = rectangles_collisions_2, plot_collisions = True,
    )

visualize_rectangles_html(
    rectangles = rectangles_3,
    secondary_rectangles = rectangles_optim_3,
    secondary_legend_title = 'optimale Trajektorie',
    hook_rectangles = rectangles_optim_hooks_3,
    connect_points = (trajectory_3[1][0], trajectory_3[-1][0]),
    hook_num = hook_num,
    hook_model = hook_model,
    trajectory_process = 3,
    html_filename = '/home/mo/Thesis/Evaluation/Trajektorientests/traj3_frame_' + str(frame) + '.html',
    collision_list = collisions_3, rectangles_collisions = rectangles_collisions_3, plot_collisions = True,
    )

visualize_rectangles_html(
    rectangles = rectangles_4,
    secondary_rectangles = rectangles_optim_4,
    secondary_legend_title = 'optimale Trajektorie',
    hook_rectangles = rectangles_optim_hooks_4,
    connect_points = (trajectory_4[1][0], trajectory_4[-1][0]),
    hook_num = hook_num,
    hook_model = hook_model,
    trajectory_process = 4,
    html_filename = '/home/mo/Thesis/Evaluation/Trajektorientests/traj4_frame_' + str(frame) + '.html',
    collision_list = collisions_4, rectangles_collisions = rectangles_collisions_4, plot_collisions = True,
    )

/tmp/ipykernel_43167/598473234.py:18: MatplotlibDeprecationWarning:

The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.



HTML-Datei '/home/mo/Thesis/Evaluation/Trajektorientests/traj1_frame__work.html' erfolgreich erstellt!
HTML-Datei '/home/mo/Thesis/Evaluation/Trajektorientests/traj2_frame__work.html' erfolgreich erstellt!
HTML-Datei '/home/mo/Thesis/Evaluation/Trajektorientests/traj3_frame__work.html' erfolgreich erstellt!
HTML-Datei '/home/mo/Thesis/Evaluation/Trajektorientests/traj4_frame__work.html' erfolgreich erstellt!


***
***
### **CSV-Import und Aufbau einer vollständigen Tabelle**

In [2270]:
def combine_measurement_csvs(folder_path, output_filename='combined_measurements.csv', ignore_hooks=None):
    if ignore_hooks is None:
        ignore_hooks = []

    all_data = []

    # Alle CSV-Dateien im Ordner durchgehen
    for file in os.listdir(folder_path):
        if file.endswith('.csv') and file != output_filename:
            filepath = os.path.join(folder_path, file)
            df = pd.read_csv(filepath)

            # Falls Spalten fehlen, über Dateiname extrahieren
            if 'hook_num' not in df.columns or 'trajectory_process' not in df.columns:
                match = re.search(r"hook(\d+)_traj(\d+)", file)
                if match:
                    hook_num = int(match.group(1))
                    trajectory_process = int(match.group(2))
                    df['hook_num'] = hook_num
                    df['trajectory_process'] = trajectory_process
            all_data.append(df)

    if all_data:
        combined_df = pd.concat(all_data, ignore_index=True)

        # Gruppieren und aggregieren
        aggregated_df = combined_df.groupby(['hook_num', 'trajectory_process'], as_index=False).agg({
            'x_max_local': 'max',
            'x_min_local': 'min',
            'y_max_local': 'max',
            'y_min_local': 'min',
            'z_max_local': 'max',
            'z_min_local': 'min',
            'roll_max': 'max',
            'pitch_max': 'max',
            'yaw_max': 'max',
            'x_mean_local': 'mean',
            'y_mean_local': 'mean',
            'z_mean_local': 'mean'
        })

        # Spaltenreihenfolge setzen
        desired_columns = [
            'hook_num',
            'trajectory_process',
            'x_max_local',
            'x_min_local',
            'y_max_local',
            'y_min_local',
            'z_max_local',
            'z_min_local',
            'roll_max',
            'pitch_max',
            'yaw_max',
            'x_mean_local',
            'y_mean_local',
            'z_mean_local'
        ]

        aggregated_df = aggregated_df[desired_columns]

        # Mittelwerte je trajectory_process berechnen
        mean_rows = []

        # for traj_proc in sorted(aggregated_df['trajectory_process'].unique()):
        for traj_proc in sorted(aggregated_df[aggregated_df['hook_num'] != -1]['trajectory_process'].unique()):
            # df_proc = aggregated_df[
            #     (aggregated_df['trajectory_process'] == traj_proc) & 
            #     (~aggregated_df['hook_num'].isin(ignore_hooks))
            # ]
            df_proc = aggregated_df[
                (aggregated_df['trajectory_process'] == traj_proc) & 
                (aggregated_df['hook_num'] != -1) &  # <-- Filter einfügen
                (~aggregated_df['hook_num'].isin(ignore_hooks))
            ]

            if not df_proc.empty:
                mean_row = df_proc.mean(numeric_only=True)
                mean_row['hook_num'] = -1  # Dummywert für Mittelwertzeile
                mean_row['trajectory_process'] = traj_proc  # Bleibt der Trajektorienprozess!
                mean_rows.append(mean_row)

        # Mittelwertzeilen anhängen
        if mean_rows:
            mean_df = pd.DataFrame(mean_rows)
            aggregated_df = pd.concat([aggregated_df, mean_df], ignore_index=True)

        # CSV speichern
        combined_path = os.path.join(folder_path, output_filename)
        aggregated_df.to_csv(combined_path, index=False, float_format="%.15g")
        print(f"Zusammengeführte Datei gespeichert unter: {combined_path}")
    else:
        print("Keine gültigen CSV-Dateien gefunden.")


In [2271]:
combine_measurement_csvs('/home/mo/Thesis/Evaluation/Trajektorientests/CSV-Exporte/Frame_' + str(frame), ignore_hooks=[])

Zusammengeführte Datei gespeichert unter: /home/mo/Thesis/Evaluation/Trajektorientests/CSV-Exporte/Frame__work/combined_measurements.csv
